In [1]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))
phase = "cross"

env = Environment(
    phase=phase,
    args=args["environment"]
)

agent = DQN(env, args[phase])

In [3]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [74]:
#env.scramble()
obs, reward, done = env.step("B'")
print(reward)
print(env.state2) 
#print(env.state2.shape)
#print(env.algorithm.status())
#print(env.cube)
print(agent.policy_net(agent.state_to_tensor(env.state2)))

2.0
[[[ 5  3  1  0  2]
  [ 0  5 -1  1 22]
  [ 0  5  3  2 19]
  [ 0 -1  4  3 20]
  [ 1 -1 -1  4  4]
  [ 5 -1  2  5 16]
  [ 0  3  4  6 17]
  [ 2  1 -1  7  7]
  [ 2  1  4  8  6]
  [-1  0  3  9 18]
  [-1  3 -1 10 10]
  [-1  1  4 11  3]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  3  4 14  9]
  [-1  2 -1 15 15]
  [-1  3  5 16 11]
  [ 4  3  1 17  0]
  [ 0  2 -1 18 24]
  [ 0  2  4 19 23]
  [ 1 -1  5 20  5]
  [ 0 -1 -1 21 21]
  [ 3 -1  1 22  1]
  [ 1  2  5 23  8]
  [ 4  2 -1 24 14]
  [ 5  0  2 25 25]]]
tensor([[ 0.1098,  0.1021,  0.1159, -0.0991, -0.0385,  0.0311,  0.1200, -0.1146,
          0.0402,  0.0614, -0.0981,  0.1053]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [5]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 12),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 12, hidden_size * 6),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 6, hidden_size * 3),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 3, hidden_size),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size, hidden_size // 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size // 2, 12)
)


In [6]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[-0.0064, -0.0893, -0.1205,  0.0524, -0.0933,  0.0781, -0.0914, -0.0879,
         -0.1321, -0.0770, -0.0699, -0.0982]], grad_fn=<AddmmBackward0>)

In [7]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[-1.8803,  0.3340, -0.6137,  ..., -0.3599,  1.2270, -0.3906],
        [-4.4939, -0.7348, -0.2796,  ..., -1.3279,  3.1091, -0.6856],
        [-7.1075, -1.8036,  0.0545,  ..., -2.2959,  4.9911, -0.9805],
        [-9.7211, -2.8725,  0.3887,  ..., -3.2640,  6.8732, -1.2755]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[ 0.0547, -0.0286],
        [ 0.6752, -0.4717],
        [ 1.2957, -0.9149],
        [ 1.9163, -1.3580]], grad_fn=<AddmmBackward0>)
